In [4]:
import pandas as pd
import zipfile
import os
import sys
from datetime import datetime
from datetime import timedelta
import numpy as np
pd.set_option('display.max_columns', 500)

In [ ]:
# dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d')
vars_serg = ["estadoOrden","Ciudad","Departamento",
             "stado_producto","Marca","Pasillo","Categoria",
             "ProductoEnPromocion","cantidadVendida",
             "TotalPagado","Domicilio","PrecioFinal",
             "FechaCreaciónOrden"]

In [ ]:
import dask.dataframe as dd

In [ ]:
df_dask = dd.read_csv("Data/MERQ_CONS",
                      usecols=vars_serg,
                      encoding='latin1',
                      assume_missing=True,
                     dtype={'estadoOrden' : str,
                            'TotalPagado' : np.float64,
                            "Domicilio" : str,
                            "FechaCreaciónOrden" : str,
                            "FechaProgramaciónOrden" : str,
                            "FechadeEntregaOrden" : str,
                            "Ciudad" : str,
                            "fechacreacionProducto" : str,
                            "stado_producto" : str,
                            "Marca" : str,
                            "Departamento" : str,
                            "Pasillo" : str,
                            "Categoria" : str,
                            "ProductoEnPromocion" : np.float64,
                            "cantidadVendida" : np.float64,
                            "PrecioFinal": np.float64})

In [ ]:
df_dask = df_dask.dropna()
df_dask.set_index("FechaCreaciónOrden")
df_dask["FechaCreaciónOrden"] = df_dask["FechaCreaciónOrden"].map_partitions(pd.to_datetime,
                                                                             format='%Y-%m-%d',
                                                                             meta = ('datetime64[ns]'))

In [ ]:
df_dask.head()

In [ ]:
vars_group_b = ["FechaCreaciónOrden",
                "Ciudad","Departamento",
                "estadoOrden","stado_producto",
                "Marca","Pasillo","Categoria",
                "ProductoEnPromocion"]

In [ ]:
agg = df_dask.groupby(vars_group_b).sum()

In [ ]:
%time nulls.compute()

In [ ]:
report_device = df_dask.estadoOrden.value_counts()/df_dask.estadoOrden.value_counts().sum()
report_device.compute()

In [ ]:
df_dask.dtypes

In [ ]:
df_p = pd.read_csv("../Data/MERQ_CONS",
                      usecols=vars_serg,
                      encoding='latin1',
                     dtype={'estadoOrden' : str,
                            'TotalPagado' : np.float64,
                            "Domicilio" : str,
                            "FechaCreaciónOrden" : str,
                            "FechaProgramaciónOrden" : str,
                            "FechadeEntregaOrden" : str,
                            "Ciudad" : str,
                            "fechacreacionProducto" : str,
                            "stado_producto" : str,
                            "Marca" : str,
                            "Departamento" : str,
                            "Pasillo" : str,
                            "Categoria" : str,
                            "ProductoEnPromocion" : np.float64,
                            "cantidadVendida" : np.float64,
                            "PrecioFinal": np.float64})

In [ ]:
df_p = df_p.dropna()

In [ ]:
agg = df_p.groupby(vars_group_b,as_index=False).sum()

In [ ]:
agg.shape

In [ ]:
agg.to_csv("Data/AGG_MERQUEO.csv",index=False)

## Agregado sub set

In [5]:
%time df = pd.read_csv("../Data/AGG_MERQUEO.csv", encoding='latin1')

CPU times: user 1.24 s, sys: 106 ms, total: 1.34 s
Wall time: 1.28 s


In [6]:
df['FechaCreaciónOrden'] = pd.to_datetime(df.FechaCreaciónOrden)

In [7]:
df.set_index("FechaCreaciónOrden",inplace=True)

In [ ]:
df_sb = df[df.Ciudad.str.contains("Bogotá")][['cantidadVendida']]
df_sb = df_sb.resample('D').sum()

In [ ]:
df.Ciudad.value_counts()